<a href="https://colab.research.google.com/github/svmk646-crypto/Manikanta/blob/main/Manikanta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded=files.upload()
print(uploaded)

Saving operators_roster.csv to operators_roster.csv
{'operators_roster.csv': b'operator_id,name,factory_id,primary_line,primary_shift,skill_level,certifications,team,hire_date,overtime_eligible,hourly_rate_eur,reliability_score\nOP-001,Alex Schmidt,FRA-PLANT-01,Line-C,Shift-2,Junior,SixSigma-Yellow,Bravo,2021-05-14,True,18.2,84.2\nOP-002,Sam M\xc3\xbcller,FRA-PLANT-01,Line-C,Shift-2,Senior,"IPC-A-610,Lockout/Tagout",Delta,2021-04-18,True,36.88,92.1\nOP-003,Taylor Keller,FRA-PLANT-01,Line-C,Shift-3,Intermediate,"ISO9001,CE-Machinery,Lean-Basics",Bravo,2023-04-23,True,27.76,89.0\nOP-004,Jordan Weber,FRA-PLANT-01,Line-A,Shift-1,Senior,"SixSigma-Green,IPC-A-610,CE-Machinery",Alpha,2023-07-10,True,40.6,91.6\nOP-005,Casey Fischer,FRA-PLANT-01,Line-B,Shift-1,Intermediate,Lean-Basics,Bravo,2024-06-11,True,31.68,84.0\nOP-006,Riley Beck,FRA-PLANT-01,Line-B,Shift-3,Senior,ISO9001,Sigma,2021-12-27,True,31.35,80.5\nOP-007,Morgan Wagner,FRA-PLANT-01,Line-A,Shift-1,Intermediate,IPC-A-610,Alpha,2021-0

# New Section

In [ ]:
pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,when,lit,sum
spark=SparkSession.builder.appName("testing_my_data").getOrCreate()
#Read All CSV file from drive
df_manitenance_event=spark.read.option("header",True).option("inferSchema",True).csv("/content/maintenance_events.csv")
df_manufacturing_event=spark.read.option("header",True).option("inferSchema",True).csv("/content/manufacturing_factory_dataset.csv")
df_operators_event=spark.read.option("header",True).option("inferSchema",True).csv("/content/operators_roster.csv")
#Task 2: Become a Data Detective
df_manitenance_event.createOrReplaceTempView("maintenance_table")
a_maintenace=spark.sql(""" select sum(cost_eur)as count,sum(downtime_min)as down_time,count(event_id)as event_count,
sum(case when reason="Unplanned Breakdown" then 1 else 0 end) as Unplanned_count,avg(downtime_min)as avg_down_time from maintenance_table  """)
#Task3.1:data cleaning to handling null values in the manufacting data frrame
df_manufacturing_event_clear=df_manufacturing_event.fillna({"planned_qty":0,"produced_qty":0,"scrap_qty":0,"defects_count":0,"energy_kwh":0})
df_manitenance_event_clear=df_manitenance_event.withColumn("parts_used",when(col("parts_used").isNull(),lit("No_Parts")).otherwise(col("parts_used")))
#task3.2:Apply Transformation
df_manufacturing_event_data=df_manufacturing_event.groupBy(col("operator_id"))\
.agg(sum(when(col("workorder_status")=="In-Progress",1).otherwise(0)).alias("IN_PROGRESS"),
sum(when(col("workorder_status")=="Planned",1).otherwise(0)).alias("Planned"),
sum(when(col("workorder_status")=="On-Hold",1).otherwise(0)).alias("On-Hold"),
sum(when(col("workorder_status")=="Completed",1).otherwise(0)).alias("Completed"))
#Task3.3 Create Fact Table
fact_df = df_manufacturing_event.alias("m") \
    .join(
        df_manitenance_event.alias("mt"),
        (col("m.factory_id") == col("mt.factory_id")) &
        (col("m.line_id") == col("mt.line_id")),
        how="left"
    ) \
    .select(
        col("m.timestamp").alias("event_timestamp"),
        col("m.factory_id"),
        col("m.line_id"),
        col("m.operator_id"),
        col("m.planned_qty"),
        col("m.produced_qty"),
        col("m.scrap_qty"),
        col("m.defects_count"),
        col("m.cycle_time_s"),
        col("m.oee"),
        col("m.availability"),
        col("m.performance"),
        col("m.quality"),
        col("mt.downtime_min"),
        col("mt.cost_eur"),
        col("m.energy_kwh")
    )
#Task 3.4 Save your fact_table in both Parquet and CSV formats.
fact_df.write.mode("overwrite").parquet("/content/fact_manufacturing_maintenance_parquet")
fact_df.write.mode("overwrite").csv("/content/fact_manufacturing_maintenance_csv")



In [ ]:
fact_df.show()

+-------------------+------------+-------+-----------+-----------+------------+---------+-------------+------------+-----+------------+-----------+-------+------------+--------+----------+
|    event_timestamp|  factory_id|line_id|operator_id|planned_qty|produced_qty|scrap_qty|defects_count|cycle_time_s|  oee|availability|performance|quality|downtime_min|cost_eur|energy_kwh|
+-------------------+------------+-------+-----------+-----------+------------+---------+-------------+------------+-----+------------+-----------+-------+------------+--------+----------+
|2025-11-03 06:00:00|FRA-PLANT-01| Line-A|     OP-015|         47|          41|        1|            0|        44.4|0.986|         1.0|      1.033|  0.954|          60|  346.87|     11.31|
|2025-11-03 06:00:00|FRA-PLANT-01| Line-A|     OP-015|         47|          41|        1|            0|        44.4|0.986|         1.0|      1.033|  0.954|         120| 1073.98|     11.31|
|2025-11-03 06:00:00|FRA-PLANT-01| Line-A|     OP-015| 

In [ ]:
df_manitenance_event_clear.show(20)


+--------------------+------------+-------+----------------+-------------------+-------------------+-------------------+------------+-------------+-----------------+--------+--------------------+-------------+
|            event_id|  factory_id|line_id|maintenance_type|             reason|         start_time|           end_time|downtime_min|technician_id|       parts_used|cost_eur|             outcome|next_due_date|
+--------------------+------------+-------+----------------+-------------------+-------------------+-------------------+------------+-------------+-----------------+--------+--------------------+-------------+
|MEV-202511031011-...|FRA-PLANT-01| Line-B|      Inspection|Planned Maintenance|2025-11-03 10:11:00|2025-11-03 10:41:00|          30|     TECH-010|PART-022,PART-040| 2201.88|Root Cause Identi...|   2025-11-06|
|MEV-202511031035-...|FRA-PLANT-01| Line-A|      Preventive|Planned Maintenance|2025-11-03 10:35:00|2025-11-03 10:50:00|          15|     TECH-012|PART-010,PART